In [1]:
import os
os.chdir('..')

In [2]:
from polygon.data import PadUfes20Data
from polygon.tasks import PadUfes20_ImageClassification_Task, Phase
from torch.utils.data import DataLoader
from torchvision.models import resnet34
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import torchvision.transforms as TF
import numpy as np

/mnt/tank/scratch/tpolevaya/miniconda3/envs/polygon/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
class ValueCollector:
    def __init__(self):
        self.values = []
    def put(self, vs):
        self.values.append(vs)
    def get(self):
        if len(self.values) == 0: 
            return []
        el = self.values[0]
        if isinstance(el, np.ndarray):
            return np.concatenate(self.values, axis=0)
        if isinstance(el, torch.Tensor):
            return torch.cat(self.values, dim=0)
        if isinstance(el, (list, tuple)):
            L = []
            for el in self.values:
                L += list(el)
            return L
        return self.values[:]
    
def mean(L:list):
    if len(L) == 0: return np.nan
    return sum(L) / len(L)

In [4]:
def get_img_tfm():
    _normalize = TF.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    return TF.Compose([TF.Resize((256,256)), TF.ToTensor(), _normalize])

In [5]:
def create_model():
    model = resnet34(weights='IMAGENET1K_V1')
    model.fc = nn.Linear(model.fc.in_features, task.get_num_classes())
    return model

In [6]:
def collect_predictions(model, dl, device, show_progress:bool=False):
    model = model.eval()
    preds_vc, ids_vc = ValueCollector(), ValueCollector()
    data_iter = tqdm(dl, leave=False) if show_progress else dl
    for batch in data_iter:
        with torch.no_grad():
            out = model(batch['image'].to(device))
        preds_vc.put(out.argmax(dim=1).detach().cpu())
        ids_vc.put(batch['ID'])
    ids, preds = ids_vc.get(), preds_vc.get()
    return dict(zip(ids, preds.numpy().tolist()))

In [7]:
def do_one_epoch(model, optimizer, loss_fn, dl, device, phase:Phase, show_progress:bool=False):
    is_training = (phase == Phase.Train)
    if is_training:
        model = model.train()
    else:
        model = model.eval()
        
    loss_vc = ValueCollector()
    data_iter = tqdm(dl, leave=False) if show_progress else dl
    for batch in data_iter:
        if is_training:
            optimizer.zero_grad()
            
        with torch.set_grad_enabled(is_training):
            out = model(batch['image'].to(device))
            loss = loss_fn(out, batch['label'].to(device))
        loss_vc.put(loss.detach().cpu().item())

        if is_training:
            loss.backward()
            optimizer.step()
            
    model = model.eval()
    print(f'Phase {phase}: loss={mean(loss_vc.get())}')

In [8]:
data = PadUfes20Data('../polygon_data')
task = PadUfes20_ImageClassification_Task(data, seed=0)

train_dl = DataLoader(task.get_dataset(Phase.Train, tfm=get_img_tfm()), batch_size=32, shuffle=True)
valid_dl = DataLoader(task.get_dataset(Phase.Valid, tfm=get_img_tfm()), batch_size=32, shuffle=False)
test_dl = DataLoader(task.get_dataset(Phase.Test, tfm=get_img_tfm()), batch_size=32, shuffle=False)

device = torch.device('cuda')
model = create_model().to(device).eval()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

num_epochs = 10
for epoch in tqdm(list(range(num_epochs)), desc='Epoch'):
    do_one_epoch(model, optimizer, loss_fn, train_dl, device, phase=Phase.Train, show_progress=True)
    do_one_epoch(model, None, loss_fn, valid_dl, device, phase=Phase.Valid, show_progress=True)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

Phase Phase.Train: loss=1.169006808237596


  0%|          | 0/15 [00:00<?, ?it/s]

Phase Phase.Valid: loss=1.104682997862498


...

Phase Phase.Valid: loss=1.5589163819948832


  0%|          | 0/44 [00:00<?, ?it/s]

Phase Phase.Train: loss=0.12883474200498313


  0%|          | 0/15 [00:00<?, ?it/s]

Phase Phase.Valid: loss=1.563395889600118


In [9]:
test_predictions = collect_predictions(model, test_dl, device)

In [10]:
task.evaluate(Phase.Test, test_predictions)

{'balanced_accuracy': 0.6014822631438033}